In [1]:
from dotenv import load_dotenv
from crewai import Crew
from tasks import MemoryTasks
from agents import LTMAgents
import os
load_dotenv()


CLEARING MEMORY


True

In [2]:
user_context = "I want to create a meal plan for myself for lunch but Im allergic to fish. Im also a vegetarian?"
subsequent_request = 'can you provide a meal plan for me?'

In [3]:
os.environ['OPENAI_API_BASE']='https://api.groq.com/openai/v1'
os.environ['OPENAI_MODEL_NAME']='llama3-70b-8192'  # Adjust based on available model
os.environ['OPENAI_API_KEY']=os.environ["GROQ_API_KEY"] 

In [4]:

tasks = MemoryTasks()
agents = LTMAgents()

# create agents
context_agent = agents.context_agent()
retrieval_agent = agents.retrieval_agent()

# create tasks
analyze_and_insert_task = tasks.analyze_and_insert_task(context_agent, user_context)
analyze_and_insert_meaningful_info_task = tasks.analyze_and_insert_meaningful_info_task(context_agent, user_context)
retrieve_and_combine_relevant_memories_task = tasks.retrieve_and_combine_relevant_memories_task(retrieval_agent, subsequent_request)
    
crew = Crew(
    agents=[
        context_agent,
        retrieval_agent
    ],
    tasks=[
        analyze_and_insert_task,
        analyze_and_insert_meaningful_info_task,
        retrieve_and_combine_relevant_memories_task
    ]
)

In [5]:
result = crew.kickoff()



> Entering new CrewAgentExecutor chain...
I understand the importance of this task and the specific format required for the final answer.

Thought: I now can give a great answer 

Final Answer: 
Dietary Restriction: Fish
Diet: Vegetarian

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: Let's start by checking with the Retrieval Specialist to see if the attribute "Dietary Restriction" already exists in the database.

Action: Ask question to co-worker
Action Input: {"coworker": "Retrieval Specialist", "question": "Does the attribute 'Dietary Restriction' already exist in the database?", "context": "Checking if the attribute 'Dietary Restriction' already exists in the database"}

> Entering new CrewAgentExecutor chain...
Thought: I need to check if the attribute 'Dietary Restriction' already exists in the database.

Action: retrieve_memories
Action Input: {"input_text": "Dietary Restriction"}Error: List index out of range.
 

[]

Let me try again.

Thought: I need

In [6]:
print(result)

Agent stopped due to iteration limit or time limit.


In [ ]:
import asyncio
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI()

def get_response(user_query, memories):

    template = """
    You are a helpful assistant. 
    Answer the following questions considering the user's question.
    You will also be provided a list of properties, preferences and restrictions correlating to the user that you should heavily consider
    when answering the user. These attributes are very important and should not be ignored

    User question: {user_question}
    properties, preferences and restrictions: {memories}
    """

    prompt = ChatPromptTemplate.from_template(template)

    llm = ChatOpenAI()
        
    chain = prompt | llm | StrOutputParser()
    
    return chain.invoke({
        "user_question": user_query,
        "memories": memories
    })


In [ ]:
print(get_response(subsequent_request, result))

In [ ]:
from memo_store import MemoStore

store = MemoStore(verbosity=0, reset=False, path_to_db_dir="./tmp/teachable_agent_db")
store.list_memos()

In [ ]:
from tools import MemoryTools

mems = MemoryTools.retrieve_memories('diet')
print(mems)

In [ ]:
from memo_store import MemoStore

memo_store = MemoStore(verbosity=0, reset=False, path_to_db_dir="./tmp/teachable_agent_db")
memo_list = memo_store.get_related_memos(
    'diet',
    n_results=10,
    threshold=1.5
)
print("Related memos:", memo_list)